In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [ ]:
p_file = "/home/matthias/projects/EfficientCN/train/softmax/affnist/results2/results.pkl"

In [ ]:
with open(p_file, "rb") as file:
    RES = pickle.load(file)

In [ ]:
scores = list(RES.keys())
scores

In [ ]:
runs = list(RES[1].keys())
runs

In [ ]:
def acu_acc_res(res):
    runs = list(res.keys())
    accs = [res[run]["acc"]["valid"] for run in runs]
    accs = np.array(accs)
    #
    accs_mu = accs.mean(axis=0)
    accs_sd = accs.std(axis=0)
    return accs_mu, accs_sd

def acu_key_res(res, key):
    runs = list(res.keys())
    vals = np.array([res[run][key] for run in runs])
    return vals.mean(axis=0), vals.std(axis=0)

def acc_norm_caps(res):
    runs = list(res.keys())
    R_norm_caps = []
    for run in runs:
        r = res[run]["norm_caps"]
        mus = r["mu"]
        stds = r["sd"]
        maxs = r["max"]
        deads = r["dead"]
        #
        r_norm_caps = []
        for idx in range(len(mus)):
            mu = mus[idx]
            sd = stds[idx]
            mx = maxs[idx]
            #
            dead = deads[idx]
            mu = mu[dead == False].mean()
            sd = sd[dead == False].mean()
            mx = mx[dead == False].mean()
            r_norm_caps.append([mu, sd, mx, 1 - dead.mean()])
        R_norm_caps.append(r_norm_caps)

    R_norm_caps = np.array(R_norm_caps)
    return R_norm_caps.mean(axis=0)

def plot_layer_vals(mu, sd=None, scores=None, ylim=None):
    n_scores, n_layers = mu.shape
    if sd is not None:
        assert mu.shape == sd.shape
    if scores is not None:
        assert len(scores) == n_scores
    else:
        scores = list(range(n_scores))
    
    fig, ax = plt.subplots(1, 1)
    for lay_idx in range(mu.shape[1]):
        ax.scatter(scores, mu[:, lay_idx], label=str(lay_idx))
        if sd is not None:  
            ax.fill_between(scores,
                        mu[:, lay_idx] - sd[:, lay_idx],
                        mu[:, lay_idx] + sd[:, lay_idx], alpha=0.2)
        ax.set_xscale("log", base=10)
    #plt.xticks(scores)
    #plt.xscale("log")
    plt.legend()
    if ylim is not None:
        plt.ylim(*ylim)
    plt.show()

# For all scores

In [ ]:
fig, ax = plt.subplots(1, 1)
for acc_mu, acc_sd, score in zip(R_scores["accs_mu"], R_scores["accs_sd"], R_scores["scores"]):
    xx = list(range(len(accs_mu)))
    ax.plot(xx, acc_mu, label=score)
    ax.fill_between(xx, acc_mu - acc_sd, acc_mu + acc_sd)
plt.legend()
plt.show()

In [ ]:
scores = scores[2:-1]
scores

In [ ]:
R_scores = {
    "dyc_mu": [],
    "dyc_sd": [],
    "bon_mu": [],
    "bon_sd": [],
    "rnd_mu": [],
    "rnd_sd": [],
    "rac_mu": [],
    "rac_sd": [],
    "norm": [],
    "scores": [],
    "dead_mu": [],
    "dead_sd": [],
    "accs_mu": [],
    "accs_sd": []
}
for score in scores:
    res = RES[score]
    dynamics_mu, dynamics_sd = acu_key_res(res, "dynamics")
    bonding_mu, bonding_sd = acu_key_res(res, "bonding")
    rnd_mu, rnd_sd = acu_key_res(res, "rnd")
    rac_mu, rac_sd = acu_key_res(res, "rac")
    norm_caps = acc_norm_caps(res)
    dead_mu, dead_sd = acu_key_res(res, "dead")
    accs_mu, accs_sd = acu_acc_res(res)
    #
    R_scores["scores"].append(score)
    R_scores["dyc_mu"].append(dynamics_mu)
    R_scores["dyc_sd"].append(dynamics_sd)
    R_scores["bon_mu"].append(bonding_mu)
    R_scores["bon_sd"].append(bonding_sd)
    R_scores["rnd_mu"].append(rnd_mu)
    R_scores["rnd_sd"].append(rnd_sd)
    R_scores["rac_mu"].append(rac_mu)
    R_scores["rac_sd"].append(rac_sd)
    R_scores["norm"].append(norm_caps)
    R_scores["dead_mu"].append(dead_mu)
    R_scores["dead_sd"].append(dead_sd)
    R_scores["accs_mu"].append(accs_mu)
    R_scores["accs_sd"].append(accs_sd)

In [ ]:
R_scores["scores"]

In [ ]:
plot_layer_vals(mu=np.array(R_scores["dyc_mu"]),
                sd=np.array(R_scores["dyc_sd"]),
                scores=R_scores["scores"],
                ylim=(0,1))

In [ ]:
plot_layer_vals(np.array(R_scores["bon_mu"]),
                np.array(R_scores["bon_sd"]),
                R_scores["scores"],
                (0,1.1))

In [ ]:
plot_layer_vals(np.array(R_scores["rnd_mu"]),
                np.array(R_scores["rnd_sd"]),
                R_scores["scores"],
                (0,1.1))

In [ ]:
plot_layer_vals(np.array(R_scores["rac_mu"]),
                np.array(R_scores["rac_sd"]),
                R_scores["scores"],
                (0,1.1))

In [ ]:
plot_layer_vals(np.array(R_scores["dead_mu"]),
                np.array(R_scores["dead_sd"]),
                R_scores["scores"],
                (-0.1,1))

In [ ]:
norms = np.array(R_scores["norm"])

In [ ]:
plot_layer_vals(norms[:,:,0],
                norms[:,:,1],
                scores=R_scores["scores"],
                ylim=(0,1))

In [ ]:
plot_layer_vals(norms[:,:,2],
                scores=R_scores["scores"],
                ylim=(0,1.1))

In [ ]:
fig, ax = plt.subplots(1, 1)
for acc_mu, acc_sd, score in zip(R_scores["accs_mu"], R_scores["accs_sd"], R_scores["scores"]):
    xx = list(range(len(accs_mu)))
    ax.plot(xx, acc_mu, label=score)
    ax.fill_between(xx, acc_mu - acc_sd, acc_mu + acc_sd)
plt.legend()
plt.show()

In [ ]:
np.array(R_scores["accs_mu"])[:, -5:]

# For one score (accumulated)

In [ ]:
score = 0.1
res = RES[score]

In [ ]:
res

In [ ]:
accs_mu, accs_sd = acu_acc_res(res)
#
xx = list(range(len(accs_mu)))
#
plt.plot(xx, accs_mu)
plt.fill_between(xx, accs_mu - accs_sd, accs_mu + accs_sd)

In [ ]:
dead_mu, dead_sd = acu_key_res(res, "dead")
dead_mu, dead_sd

In [ ]:
rnd_mu, rnd_sd = acu_key_res(res, "rnd")
rnd_mu, rnd_sd

In [ ]:
rac_mu, rac_sd = acu_key_res(res, "rac")
rac_mu, rac_sd

In [ ]:
racnd_mu, racnd_sd = acu_key_res(res, "racnd")
racnd_mu, racnd_sd

In [ ]:
bonding_mu, bonding_sd = acu_key_res(res, "bonding")
bonding_mu, bonding_sd

In [ ]:
dynamics_mu, dynamics_sd = acu_key_res(res, "dynamics")
dynamics_mu, dynamics_sd

In [ ]:
norm_caps = acc_norm_caps(res)
norm_caps

In [ ]:
AS